# Introduction
In this guide, we will walk you through building a Retrieval Augmented Generation (RAG) application using Couchbase Capella as the database, [Mistral-7B-Instruct-v0.3](https://build.nvidia.com/mistralai/mistral-7b-instruct-v03/modelcard) model as the large language model provided by Capella Model Services. We will use the [NVIDIA NeMo Retriever Llama3.2](https://build.nvidia.com/nvidia/llama-3_2-nv-embedqa-1b-v2/modelcard) model for generating embeddings via Capella Model Services.

This notebook demonstrates how to build a RAG system using:
- The [BBC News dataset](https://huggingface.co/datasets/RealTimeData/bbc_news_alltime) containing news articles
- Couchbase Capella as the vector store with **Hyperscale and Composite Vector Indexes** for high-performance vector search
- Capella Model Services for embeddings and text generation
- LangChain framework for the RAG pipeline

We leverage Couchbase's **Query Service** to create and manage Hyperscale Vector Indexes, enabling efficient semantic search capabilities that can scale to billions of vectors. Hyperscale and Composite indexes provide superior performance for large-scale vector search operations compared to traditional approaches. This tutorial can also be recreated using the Search Service with [Search Vector Index](https://github.com/couchbase-examples/vector-search-cookbook/blob/main/capella-model-services/langchain/search_based/RAG_with_Capella_Model_Services_and_LangChain.ipynb).

**Key Features:**
- High-performance vector search using Hyperscale/Composite indexes
- Performance benchmarks showing optimization benefits
- Complete RAG workflow with caching optimization

**Requirements:** Couchbase Server 8.0+ or Capella with Query Service enabled.

Semantic search goes beyond simple keyword matching by understanding the context and meaning behind the words in a query, making it an essential tool for applications that require intelligent information retrieval. This tutorial will equip you with the knowledge to create a fully functional RAG system using Capella Model Services and [LangChain](https://langchain.com/)

# How to run this tutorial

This tutorial is available as a Jupyter Notebook (`.ipynb` file) that you can run interactively. You can access the original notebook [here](https://github.com/couchbase-examples/vector-search-cookbook/blob/main/capella-model-services/langchain/query_based/RAG_with_Capella_Model_Services_and_LangChain.ipynb)

You can either download the notebook file and run it on [Google Colab](https://colab.research.google.com/) or run it on your system by setting up the Python environment.

# Before you start

## Create and Deploy Your Operational cluster on Capella

To get started with Couchbase Capella, create an account and use it to deploy an operational cluster.

To know more, please follow the [instructions](https://docs.couchbase.com/cloud/get-started/create-account.html).


### Couchbase Capella Configuration

When running Couchbase using [Capella](https://cloud.couchbase.com/sign-in), the following prerequisites need to be met:

* Have a multi-node Capella cluster running the **Data, Query, and Index services** (Query Service is required for Hyperscale/Composite indexes).
* Create the [database credentials](https://docs.couchbase.com/cloud/clusters/manage-database-users.html) to access the bucket (Read and Write) used in the application.
* [Allow access](https://docs.couchbase.com/cloud/clusters/allow-ip-address.html) to the Cluster from the IP on which the application is running.

### Deploy Models

In order to create the RAG application, we need an embedding model to ingest the documents for Vector Search and a large language model (LLM) for generating the responses based on the context.

Capella Model Service allows you to create both the embedding model and the LLM in the same VPC as your database. There are multiple options for both the Embedding & Large Language Models, along with Value Adds to the models.

Create the models using the Capella Model Services interface. While creating the model, it is possible to cache the responses (both standard and semantic cache) and apply guardrails to the LLM responses.

For more details, please refer to the [documentation](https://docs.couchbase.com/ai/build/model-service/model-service.html). These models are compatible with the [LangChain OpenAI integration](https://python.langchain.com/api_reference/openai/index.html).

After the models are deployed, please create the API keys for them and whitelist the keys on the IP on which the tutorial is being run. For more details, please refer to the documentation on [generating the API keys](https://docs.couchbase.com/ai/api-guide/api-start.html#model-service-keys).

# Installing Necessary Libraries
To build our RAG system, we need a set of libraries. The libraries we install handle everything from connecting to databases to performing AI tasks. Each library has a specific role: Couchbase libraries manage database operations, LangChain handles AI model integrations, and we will use the OpenAI SDK for generating embeddings and calling the LLM in Capella Model services. By setting up these libraries, we ensure our environment is equipped to handle the tasks required for RAG.

In [1]:
%pip install --quiet datasets==4.4.1 langchain-couchbase==1.0.0 langchain-openai==1.1.0

Note: you may need to restart the kernel to use updated packages.


# Importing Necessary Libraries
The script starts by importing a series of libraries required for various tasks, including handling JSON, logging, time tracking, Couchbase connections, embedding generation, and dataset loading. These libraries provide essential functions for working with data, managing database connections, and processing machine learning models.

Note that we import `CouchbaseQueryVectorStore` along with `DistanceStrategy` and `IndexType` for creating Hyperscale/Composite Vector Indexes.

In [ ]:
import logging
import os
import time

from datetime import timedelta

from dotenv import load_dotenv

from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.exceptions import CouchbaseException
from couchbase.options import ClusterOptions

from datasets import load_dataset

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_couchbase.vectorstores import CouchbaseQueryVectorStore
from langchain_couchbase.vectorstores import DistanceStrategy, IndexType
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from tqdm import tqdm

# Loading Environment Variables

This notebook loads configuration from a `.env` file in the same directory. Create a `.env` file with the following variables:

**Required (no defaults):**
- `CB_CONNECTION_STRING` - Your Couchbase connection string
- `CB_USERNAME` - Couchbase database username
- `CB_PASSWORD` - Couchbase database password
- `CB_BUCKET_NAME` - Name of your Couchbase bucket
- `CAPELLA_MODEL_SERVICES_ENDPOINT` - Capella Model Services endpoint (include `/v1` suffix)
- `LLM_API_KEY` - API key for the LLM model
- `EMBEDDING_API_KEY` - API key for the embedding model

**Optional (with defaults):**
- `SCOPE_NAME` - Scope name (default: `_default`)
- `COLLECTION_NAME` - Collection name (default: `langchain_docs`)
- `LLM_MODEL_NAME` - LLM model name (default: `mistralai/mistral-7b-instruct-v0.3`)
- `EMBEDDING_MODEL_NAME` - Embedding model name (default: `nvidia/llama-3.2-nv-embedqa-1b-v2`)

> **Note:** The Capella Model Services Endpoint requires `/v1` suffix if not shown on the UI.

> If the models are running in the same region, either API key can be used interchangeably. See [generating API keys](https://docs.couchbase.com/ai/api-guide/api-start.html#model-service-keys).

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Couchbase connection settings (no defaults for sensitive values)
CB_CONNECTION_STRING = os.getenv("CB_CONNECTION_STRING")
CB_USERNAME = os.getenv("CB_USERNAME")
CB_PASSWORD = os.getenv("CB_PASSWORD")
CB_BUCKET_NAME = os.getenv("CB_BUCKET_NAME")

# Collection settings (with sensible defaults)
SCOPE_NAME = os.getenv("SCOPE_NAME", "_default")
COLLECTION_NAME = os.getenv("COLLECTION_NAME", "langchain_docs")

# Capella Model Services settings
CAPELLA_MODEL_SERVICES_ENDPOINT = os.getenv("CAPELLA_MODEL_SERVICES_ENDPOINT")

# Model names (with defaults matching tutorial recommendations)
LLM_MODEL_NAME = os.getenv("LLM_MODEL_NAME", "mistralai/mistral-7b-instruct-v0.3")
EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL_NAME", "nvidia/llama-3.2-nv-embedqa-1b-v2")

# API keys (no defaults for sensitive values)
LLM_API_KEY = os.getenv("LLM_API_KEY")
EMBEDDING_API_KEY = os.getenv("EMBEDDING_API_KEY")

# Validate required environment variables
if not all([
    CB_CONNECTION_STRING,
    CB_USERNAME,
    CB_PASSWORD,
    CB_BUCKET_NAME,
    CAPELLA_MODEL_SERVICES_ENDPOINT,
    LLM_API_KEY,
    EMBEDDING_API_KEY,
]):
    raise ValueError(
        "Missing required environment variables. Please ensure your .env file contains:\n"
        "- CB_CONNECTION_STRING\n"
        "- CB_USERNAME\n"
        "- CB_PASSWORD\n"
        "- CB_BUCKET_NAME\n"
        "- CAPELLA_MODEL_SERVICES_ENDPOINT\n"
        "- LLM_API_KEY\n"
        "- EMBEDDING_API_KEY"
    )

print("Environment variables loaded successfully")

# Connecting to the Couchbase Cluster
Couchbase will serve as our primary data store, handling all the storage and retrieval operations required for our RAG system. By establishing this connection, we enable our application to interact with the database, allowing us to perform operations such as storing embeddings, querying data, and managing collections.

In [ ]:
try:
    auth = PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)
    options = ClusterOptions(auth)
    cluster = Cluster(CB_CONNECTION_STRING, options)
    cluster.wait_until_ready(timedelta(seconds=5))
    print("Successfully connected to Couchbase")
except Exception as e:
    raise ConnectionError(f"Failed to connect to Couchbase: {str(e)}")

# Setting Up Collections in Couchbase
In Couchbase, data is organized in buckets, which can be further divided into scopes and collections. Think of a collection as a table in a traditional SQL database. Before we can store any data, we need to ensure that our collections exist. If they don't, we must create them. This step is important because it prepares the database to handle the specific types of data our application will process. By setting up collections, we define the structure of our data storage, which is essential for efficient data retrieval and management.

Moreover, setting up collections allows us to isolate different types of data within the same bucket, providing a more organized and scalable data structure. This is particularly useful when dealing with large datasets, as it ensures that related data is stored together, making it easier to manage and query. Here, we also set up the primary index for query operations on the collection and clear the existing documents in the collection if any. If you do not want to do that, please skip this step.

In [ ]:
def setup_collection(cluster, bucket_name, scope_name, collection_name, flush_collection=False):
    try:
        bucket = cluster.bucket(bucket_name)
        bucket_manager = bucket.collections()

        # Check if scope exists, create if it doesn't
        scopes = bucket_manager.get_all_scopes()
        scope_exists = any(scope.name == scope_name for scope in scopes)
        
        if not scope_exists:
            print(f"Scope '{scope_name}' does not exist. Creating it...")
            bucket_manager.create_scope(scope_name)
            print(f"Scope '{scope_name}' created successfully.")
        else:
            print(f"Scope '{scope_name}' already exists. Skipping creation.")
        
        # Check if collection exists, create if it doesn't
        collections = bucket_manager.get_all_scopes()
        collection_exists = any(
            scope.name == scope_name
            and collection_name in [col.name for col in scope.collections]
            for scope in collections
        )

        if not collection_exists:
            print(f"Collection '{collection_name}' does not exist. Creating it...")
            bucket_manager.create_collection(scope_name, collection_name)
            print(f"Collection '{collection_name}' created successfully.")
        else:
            print(f"Collection '{collection_name}' already exists. Skipping creation.")

        collection = bucket.scope(scope_name).collection(collection_name)
        time.sleep(2)  # Give the collection time to be ready for queries

        # Ensure primary index exists (required for Query Service operations)
        try:
            cluster.query(
                f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{bucket_name}`.`{scope_name}`.`{collection_name}`"
            ).execute()
            print("Primary index present or created successfully.")
        except Exception as e:
            logging.warning(f"Error creating primary index: {str(e)}")

        if flush_collection:
            # Clear all documents in the collection
            try:
                query = f"DELETE FROM `{bucket_name}`.`{scope_name}`.`{collection_name}`"
                cluster.query(query).execute()
                print("All documents cleared from the collection.")
            except Exception as e:
                print(
                    f"Error while clearing documents: {str(e)}. The collection might be empty."
                )

    except Exception as e:
        raise Exception(f"Error setting up collection: {str(e)}")


setup_collection(cluster, CB_BUCKET_NAME, SCOPE_NAME, COLLECTION_NAME, flush_collection=True)

# Load the BBC News Dataset
To build a RAG engine, we need data to search through. We use the [BBC Realtime News dataset](https://huggingface.co/datasets/RealTimeData/bbc_news_alltime), a dataset with up-to-date BBC news articles grouped by month. This dataset contains articles that were created after the LLM was trained. It will showcase the use of RAG to augment the LLM.

The BBC News dataset's varied content allows us to simulate real-world scenarios where users ask complex questions, enabling us to fine-tune our RAG's ability to understand and respond to various types of queries.

In [ ]:
try:
    news_dataset = load_dataset('RealTimeData/bbc_news_alltime', '2024-12', split="train")
    print(f"Loaded the BBC News dataset with {len(news_dataset)} rows")
except Exception as e:
    raise ValueError(f"Error loading BBC dataset: {str(e)}")

## Preview the Data

In [ ]:
print(news_dataset[:5])

## Cleaning up the Data

We will use the content of the news articles for our RAG system.

The dataset contains a few duplicate records. We are removing them to avoid duplicate results in the retrieval stage of our RAG system.

In [ ]:
news_articles = news_dataset["content"]
unique_articles = set()
for article in news_articles:
    if article:
        unique_articles.add(article)
unique_news_articles = list(unique_articles)
print(f"We have {len(unique_news_articles)} unique articles in our database.")

# Creating Embeddings using Capella Model Service
Embeddings are at the heart of semantic search. They are numerical representations of text that capture the semantic meaning of the words and phrases. Unlike traditional keyword-based search, which looks for exact matches, embeddings allow our search engine to understand the context and nuances of language, enabling it to retrieve documents that are semantically similar to the query, even if they don't contain the exact keywords. By creating embeddings using Capella Model service, we equip our RAG system with the ability to understand and process natural language in a way that is much closer to how humans understand language. This step transforms our raw text data into a format that the Capella vector store can use to find and rank relevant documents.

We are using the OpenAI Embeddings via the [LangChain OpenAI provider](https://python.langchain.com/docs/integrations/providers/openai/) with a few extra parameters specific to the Capella Model Services such as disabling the tokenization and handling of longer inputs using the LangChain handler. We provide the model and api_key and the URL for the SDK to those for Capella Model Services. For this tutorial, we are using the [nvidia/llama-3.2-nv-embedqa-1b-v2](https://build.nvidia.com/nvidia/llama-3_2-nv-embedqa-1b-v2) embedding model. If you are using a different model, you would need to change the model name accordingly.

In [ ]:
try:
    embeddings = OpenAIEmbeddings(
        openai_api_key=EMBEDDING_API_KEY,
        openai_api_base=CAPELLA_MODEL_SERVICES_ENDPOINT,
        check_embedding_ctx_length=False,
        tiktoken_enabled=False,
        model=EMBEDDING_MODEL_NAME,
    )
    print("Successfully created CapellaAIEmbeddings")
except Exception as e:
    raise ValueError(f"Error creating CapellaAIEmbeddings: {str(e)}")

# Testing the Embeddings Model
We can test the embeddings model by generating an embedding for a string using the LangChain OpenAI package

In [ ]:
print(len(embeddings.embed_query("this is a test sentence")))

# Setting Up the Couchbase Query Vector Store
The vector store is set up to store the documents from the dataset. We use `CouchbaseQueryVectorStore` which enables Hyperscale and Composite Vector Indexes for high-performance vector storage and retrieval using Couchbase's Query Service.

**Key differences from Search Vector Store:**
- Uses Query Service instead of Search Service
- Supports Hyperscale indexes that can scale to billions of vectors
- Index is created programmatically after data ingestion
- No need for a separate JSON index definition file

In [ ]:
try:
    vector_store = CouchbaseQueryVectorStore(
        cluster=cluster,
        bucket_name=CB_BUCKET_NAME,
        scope_name=SCOPE_NAME,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        distance_metric=DistanceStrategy.COSINE
    )
    print("Successfully created Couchbase Query Vector Store")
except Exception as e:
    raise ValueError(f"Failed to create vector store: {str(e)}")

# Saving Data to the Vector Store
With the Vector store set up, the next step is to populate it with data. We save the BBC articles dataset to the vector store. For each document, we will generate the embeddings for the article to use with the semantic search using LangChain.

**Important:** With Hyperscale/Composite indexes, data must be ingested BEFORE creating the index. The index creation process analyzes existing vectors to optimize search performance through clustering and quantization.

Some articles may exceed the embedding model's maximum token limit (8192 tokens). These documents are automatically skipped during ingestion. For production use, consider splitting longer documents into chunks.

In [ ]:
skipped_count = 0
ingested_count = 0

for article in tqdm(unique_news_articles, desc="Ingesting articles"):
    try:
        documents = [Document(page_content=article)]
        vector_store.add_documents(documents=documents)
        ingested_count += 1
    except Exception as e:
        # Skip documents that exceed token limits or have other issues
        if "exceeds maximum" in str(e) or "token" in str(e).lower():
            skipped_count += 1
            continue
        else:
            print(f"Failed to save document: {str(e)[:100]}...")
            skipped_count += 1
            continue

print(f"\nIngestion complete: {ingested_count} documents ingested, {skipped_count} skipped (exceeded token limit)")

# Vector Search Performance Testing

Now let's demonstrate the performance benefits of Hyperscale Vector Index by testing pure vector search performance. We'll compare:

1. **Baseline Performance**: Vector search without Hyperscale index optimization
2. **Hyperscale-Optimized Performance**: Same search with Hyperscale index
3. **Cache Benefits**: Show how caching can be applied on top of Hyperscale for repeated queries

## Vector Index Types Overview

Before we start testing, let's understand the index types available:

**BHIVE (Hyperscale) Vector Indexes:**
- **Best for**: Pure vector searches - content discovery, recommendations, semantic search
- **Performance**: High performance with low memory footprint, designed to scale to billions of vectors
- **Optimization**: Optimized for concurrent operations, supports simultaneous searches and inserts
- **Use when**: You primarily perform vector-only queries without complex scalar filtering

**Composite Vector Indexes:**
- **Best for**: Filtered vector searches that combine vector search with scalar value filtering
- **Performance**: Efficient pre-filtering where scalar attributes reduce the vector comparison scope
- **Use when**: Your queries combine vector similarity with scalar filters that eliminate large portions of data

For this tutorial, we'll test both index types to demonstrate their capabilities.

For more information, see [Couchbase Hyperscale and Composite Vector Index Documentation](https://docs.couchbase.com/cloud/vector-index/use-vector-indexes.html).

## Vector Search Test Function

In [ ]:
def test_vector_search_performance(vector_store, query, label="Vector Search"):
    """Test pure vector search performance and return timing metrics"""
    print(f"\n[{label}] Testing vector search performance")
    print(f"[{label}] Query: '{query}'")
    
    start_time = time.time()
    
    try:
        results = vector_store.similarity_search_with_score(query, k=3)
        end_time = time.time()
        search_time = end_time - start_time
        
        print(f"[{label}] Vector search completed in {search_time:.4f} seconds")
        print(f"[{label}] Found {len(results)} documents")
        
        if results:
            doc, distance = results[0]
            print(f"[{label}] Top result distance: {distance:.6f} (lower = more similar)")
            preview = doc.page_content[:100] + "..." if len(doc.page_content) > 100 else doc.page_content
            print(f"[{label}] Top result preview: {preview}")
        
        return search_time
    except Exception as e:
        print(f"[{label}] Vector search failed: {str(e)}")
        return None

## Test 1: Baseline Performance (No Hyperscale Index)

In [ ]:
test_query = "What was Pep Guardiola's reaction to Manchester City's current form?"
print("Testing baseline vector search performance without Hyperscale index optimization...")
baseline_time = test_vector_search_performance(vector_store, test_query, "Baseline Search")
print(f"\nBaseline vector search time (without Hyperscale index): {baseline_time:.4f} seconds\n")

# Creating BHIVE (Hyperscale) Vector Index

Now let's create a BHIVE vector index to enable high-performance vector searches. The index creation is done programmatically through the vector store.

**Index Configuration:**
- `index_type`: `IndexType.BHIVE` for pure vector search, `IndexType.COMPOSITE` for filtered searches
- `index_name`: Unique name for the index
- `index_description`: Controls centroids and quantization settings
  - `IVF,SQ8`: Auto-selected centroids with 8-bit scalar quantization (recommended default)
  - See [Quantization & Centroid Settings](https://docs.couchbase.com/cloud/vector-index/hyperscale-vector-index.html#algo_settings) for more options

In [ ]:
print("Creating BHIVE (Hyperscale) vector index...")
try:
    vector_store.create_index(
        index_type=IndexType.BHIVE,
        index_name="langchain_bhive_index",
        index_description="IVF,SQ8"
    )
    print("BHIVE vector index created successfully")
    
    # Wait for index to become available
    print("Waiting for index to become available...")
    time.sleep(5)
    
except Exception as e:
    if "already exists" in str(e).lower():
        print("BHIVE vector index already exists, proceeding...")
    else:
        print(f"Error creating BHIVE vector index: {str(e)}")

## Test 2: BHIVE (Hyperscale) Optimized Performance

In [ ]:
print("Testing vector search performance with BHIVE (Hyperscale) optimization...")
bhive_time = test_vector_search_performance(vector_store, test_query, "BHIVE")

# Creating Composite Vector Index

Now let's create a Composite vector index to demonstrate filtered vector searches. Composite indexes are optimized for queries that combine vector similarity with scalar filters.

In [ ]:
print("Creating Composite vector index...")
try:
    vector_store.create_index(
        index_type=IndexType.COMPOSITE,
        index_name="langchain_composite_index",
        index_description="IVF,SQ8"
    )
    print("Composite vector index created successfully")
    
    # Wait for index to become available
    print("Waiting for index to become available...")
    time.sleep(5)
    
except Exception as e:
    if "already exists" in str(e).lower():
        print("Composite vector index already exists, proceeding...")
    else:
        print(f"Error creating Composite vector index: {str(e)}")

## Test 3: Composite Index Performance

In [ ]:
print("Testing vector search performance with Composite index...")
composite_time = test_vector_search_performance(vector_store, test_query, "Composite")

## Performance Summary

In [ ]:
print("\n" + "="*60)
print("PERFORMANCE SUMMARY")
print("="*60)

print(f"Baseline Search Time:     {baseline_time:.4f} seconds")

if baseline_time and bhive_time:
    speedup = baseline_time / bhive_time if bhive_time > 0 else float('inf')
    percent_improvement = ((baseline_time - bhive_time) / baseline_time) * 100 if baseline_time > 0 else 0
    print(f"BHIVE Search Time:        {bhive_time:.4f} seconds ({speedup:.2f}x faster, {percent_improvement:.1f}% improvement)")

if baseline_time and composite_time:
    speedup = baseline_time / composite_time if composite_time > 0 else float('inf')
    percent_improvement = ((baseline_time - composite_time) / baseline_time) * 100 if baseline_time > 0 else 0
    print(f"Composite Search Time:    {composite_time:.4f} seconds ({speedup:.2f}x faster, {percent_improvement:.1f}% improvement)")

if bhive_time and composite_time:
    print("\n" + "-"*60)
    print("Index Comparison:")
    print("-"*60)
    print("- BHIVE (Hyperscale): Best for pure vector searches, scales to billions of vectors")
    print("- Composite: Best for filtered searches combining vector + scalar filters")

# Perform Semantic Search
Semantic search in Couchbase involves converting queries and documents into vector representations using an embeddings model. These vectors capture the semantic meaning of the text and are stored directly in Couchbase. When a query is made, Couchbase performs a similarity search by comparing the query vector against the stored document vectors.

With Hyperscale indexes, the similarity metric (COSINE) is configured at vector store initialization time via `distance_metric=DistanceStrategy.COSINE`. The search process uses the Query Service for efficient ANN (Approximate Nearest Neighbor) search.

**Distance Interpretation**: In vector search using Hyperscale indexes, lower distance values indicate higher similarity, while higher distance values indicate lower similarity.

In [ ]:
query = "What was Pep Guardiola's reaction to Manchester City's current form?"

try:
    # Perform the semantic search
    start_time = time.time()
    search_results = vector_store.similarity_search_with_score(query, k=5)
    search_elapsed_time = time.time() - start_time

    # Display search results
    print(
        f"\nSemantic Search Results (completed in {search_elapsed_time:.2f} seconds):"
    )
    for doc, score in search_results:
        print(f"Score: {score:.4f}, ID: {doc.id}, Text: {doc.page_content[:200]}...")
        print("---"*20)

except CouchbaseException as e:
    raise RuntimeError(f"Error performing semantic search: {str(e)}")
except Exception as e:
    raise RuntimeError(f"Unexpected error: {str(e)}")

# Retrieval-Augmented Generation (RAG) with Couchbase and LangChain
Couchbase and LangChain can be seamlessly integrated to create RAG (Retrieval-Augmented Generation) chains, enhancing the process of generating contextually relevant responses. In this setup, Couchbase serves as the vector store, where embeddings of documents are stored. When a query is made, LangChain retrieves the most relevant documents from Couchbase by comparing the query's embedding with the stored document embeddings using our Hyperscale-optimized search. These documents, which provide contextual information, are then passed to a large language model using LangChain.

The language model, equipped with the context from the retrieved documents, generates a response that is both informed and contextually accurate. This integration allows the RAG chain to leverage Couchbase's efficient storage and retrieval capabilities with Hyperscale performance, while the LLM handles the generation of responses based on the context provided by the retrieved documents.

## Using the Large Language Model (LLM) in Capella Model Services
We'll be using the [mistralai/mistral-7b-instruct-v0.3](https://build.nvidia.com/mistralai/mistral-7b-instruct-v03) large language model via the Capella Model Services inside the same network as the Capella operational database to process user queries and generate meaningful responses.

In [ ]:
try:
    llm = ChatOpenAI(openai_api_base=CAPELLA_MODEL_SERVICES_ENDPOINT, openai_api_key=LLM_API_KEY, model=LLM_MODEL_NAME, temperature=0)
    logging.info("Successfully created the Chat model in Capella Model Services")
except Exception as e:
    raise ValueError(f"Error creating Chat model in Capella Model Services: {str(e)}")

In [ ]:
llm.invoke("What was Pep Guardiola's reaction to Manchester City's current form?")

## Building the RAG Chain

In [ ]:
template = """You are a helpful bot. If you cannot answer based on the context provided, respond with a generic answer. Answer the question as truthfully as possible using the context below:
    {context}
    Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)
rag_chain = (
    {"context": vector_store.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
logging.info("Successfully created RAG chain")

In [ ]:
# Get responses
query = "What was Pep Guardiola's reaction to Manchester City's recent form?"
try:
    start_time = time.time()
    rag_response = rag_chain.invoke(query)
    rag_elapsed_time = time.time() - start_time

    print(f"RAG Response: {rag_response}")
    print(f"RAG response generated in {rag_elapsed_time:.2f} seconds")
except Exception as e:
    print("Error occurred:", e)

# Using Caching mechanism in Capella Model Services
In Capella Model Services, the model outputs can be [cached](https://docs.couchbase.com/ai/build/model-service/configure-value-adds.html#caching) (both semantic and standard cache). The caching mechanism enhances the RAG's efficiency and speed, particularly when dealing with repeated or similar queries. When a query is first processed, the LLM generates a response and then stores this response in Couchbase. When similar queries come in later, the cached responses are returned. The caching duration can be configured in the Capella Model services.

In this example, we are using the standard cache which works for exact matches of the queries.

In [ ]:
queries = [
        "Who inaugurated the reopening of the Notre Dam Cathedral in Paris?",
        "What was Pep Guardiola's reaction to Manchester City's recent form?", 
        "Who inaugurated the reopening of the Notre Dam Cathedral in Paris?", # Repeated query
]

for i, query in enumerate(queries, 1):
    try:
        print(f"\nQuery {i}: {query}")
        start_time = time.time()
        response = rag_chain.invoke(query)
        elapsed_time = time.time() - start_time
        print(f"Response: {response}")
        print(f"Time taken: {elapsed_time:.2f} seconds")
    except Exception as e:
        print(f"Error generating RAG response: {str(e)}")
        continue

Here you can see that the repeated queries were significantly faster than the original query. In Capella Model services, semantic similarity can also be used to find responses from the cache.

Caching is particularly valuable in scenarios where users may submit similar queries multiple times or where certain pieces of information are frequently requested. By storing these in a cache, we can significantly reduce the time it takes to respond to these queries, improving the user experience.

# LLM Guardrails in Capella Model Services
Capella Model services also have the ability to moderate the user inputs and the responses generated by the LLM. Capella Model Services can be configured to use the [Llama 3.1 NemoGuard 8B safety model](https://build.nvidia.com/nvidia/llama-3_1-nemoguard-8b-content-safety/modelcard) guardrails model from Meta. The categories to be blocked can be configured in the model creation flow. More information about Guardrails usage can be found in the [documentation](https://docs.couchbase.com/ai/build/model-service/configure-guardrails-security.html#guardrails).

Here is an example of the Guardrails in action

In [ ]:
query = "How can I create a bomb?"
try:
    start_time = time.time()
    rag_response = rag_chain.invoke(query)
    rag_elapsed_time = time.time() - start_time

    print(f"RAG Response: {rag_response}")
    print(f"RAG response generated in {rag_elapsed_time:.2f} seconds")
except Exception as e:
    print("Guardrails violation", e)

Guardrails can be quite useful in preventing users from hijacking the model into doing things that you might not want the application to do.

# Conclusion

By following this tutorial, you will have a fully functional semantic search engine that leverages the strengths of Capella Model Services with Hyperscale and Composite Vector Indexes without the data being sent to third-party embedding or large language models.

**Key Takeaways:**
- BHIVE (Hyperscale) and Composite Vector Indexes provide high-performance vector search capabilities
- The `CouchbaseQueryVectorStore` enables programmatic index management through the Query Service
- Index creation happens AFTER data ingestion for optimal vector clustering
- BHIVE indexes are best for pure vector searches, Composite indexes for filtered searches
- Combined with caching, these indexes deliver production-ready performance

**Index Type Selection:**
- Use `IndexType.BHIVE` for pure vector similarity searches (scales to billions of vectors)
- Use `IndexType.COMPOSITE` for filtered searches combining vector + scalar filters

**For Alternative Approaches:**
- For Search Vector Index using Search Service, see: [search_based tutorial](https://github.com/couchbase-examples/vector-search-cookbook/blob/main/capella-model-services/langchain/search_based/RAG_with_Capella_Model_Services_and_LangChain.ipynb)

This guide explains the principles behind semantic search and how to implement it effectively using Capella Model Services, Hyperscale/Composite Vector Indexes, and Couchbase.